In [ ]:
# Tesseract must be installed on your machine for this to work and added to the PATH
!pip install pytesseract

In [36]:
# Importing, the first comment can be crucial for the code to work (if something breaks, try it)

import pytesseract
import cv2
# pytesseract.pytesseract.tesseract_cmd = r'D:\\Tesseract\\tesseract.exe'

# # Load an example image
# image = cv2.imread('example.png')

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Use pytesseract to do OCR on the image
# text = pytesseract.image_to_string(gray)

# # Display the image and the extracted text
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.title('Extracted Text: ' + text)
# plt.axis('off')
# plt.show()

In [37]:
# Trying to detect whether image contains only text or not
# Not ok now, ideas will be appreciated

from PIL import Image

def contains_non_black_white_pixels(image_path):
    """
    Checks if an image contains pixels other than black or white.

    :param image_path: Path to the image file.
    :return: True if there are pixels other than black or white, False otherwise.
    """
    try:
        # Open the image
        img = Image.open(image_path).convert("RGB")  # Ensure it's in RGB mode
        
        # Iterate through the pixels
        width, height = img.size
        for x in range(width):
            for y in range(height):
                r, g, b = img.getpixel((x, y))
                # Check if the pixel is not black or white
                if (r, g, b) != (0, 0, 0) and (r, g, b) != (255, 255, 255):
                    return True  # Found a non-black/white pixel
        
        return False  # All pixels are black or white
    except Exception as e:
        print(f"Error: {e}")
        return False

# Example usage
image_path = "example2.png"  # Replace with your image path
if contains_non_black_white_pixels(image_path):
    print("The image contains pixels that are not black or white.")
else:
    print("The image contains only black and white pixels.")

The image contains pixels that are not black or white.


In [ ]:
#All images are examined whether A., B., C., and D. patterns are present. If all patterns are present, the image is moved to subfolder, called "valid".

import os
import shutil

def check_patterns_in_image(image_path):
    """
    Checks if the image contains patterns "A.", "B.", "C." and "D.".

    :param image_path: Path to the image file.
    :return: Dictionary with patterns as keys and boolean values indicating presence.
    """
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use pytesseract to do OCR on the image
    text = pytesseract.image_to_string(gray)
    
    # Check for patterns
    patterns = ["A.", "B.", "C.", "D."]
    result = {pattern: (pattern in text) for pattern in patterns}
    
    return result

def check_patterns_in_directory(directory_path):
    results = {}
    for root, _, files in os.walk(directory_path):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                image_path = os.path.join(root, filename)
                
                index = files.index(filename) + 1
                all_files = len(files)
                print(f"{index}/{all_files}")
                
                print(image_path, "is being processed")
                results[image_path] = check_patterns_in_image(image_path)
                print(f"result for ${image_path}: ", results[image_path])
    
    return results

directory_path = 'D:\\pdf_processing_extraction\\dataset'  # TODO Directory for the image (from the same language)
valid_folder = os.path.join(directory_path, "valid")
# Clear the valid folder with a warning
if os.path.exists(valid_folder):
    print(f"Warning: The folder {valid_folder} will be emptied.")
    for filename in os.listdir(valid_folder):
        file_path = os.path.join(valid_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")
os.makedirs(valid_folder, exist_ok=True)

results = check_patterns_in_directory(directory_path)
# for filename, result in results.items():
#     print(f"File: {filename}")
#     for pattern, found in result.items():
#         print(f"  Pattern '{pattern}': {'Found' if found else 'Not Found'}")

# Move files with all patterns found to the "valid" subfolder
for image_path, result in results.items():
    if all(result.values()):
        filename = os.path.basename(image_path)
        subfolder_name = os.path.basename(os.path.dirname(image_path))[-3:]
        subfolder_path = os.path.join(valid_folder, subfolder_name)
        os.makedirs(subfolder_path, exist_ok=True)
        
        dst_path = os.path.join(subfolder_path, filename)
        shutil.copy(image_path, dst_path)
        print(f"Moved {filename} to {valid_folder}")

In [ ]:
# Finding the text in the images and saving them to the text files together with the answer from any of the JSON files in the directory.
# TODO Sould we examine for strange characters left from images?

import os
import json

def search_image_in_json(image_id, json_directory):
    """
    Searches for the image name in all JSON files in the specified directory and prints the file name
    together with the respective answer field value from the JSON.

    :param image_name: Name of the image to search for.
    :param json_directory: Directory containing the JSON files.
    """
    for root, _, files in os.walk(json_directory):
        for file in files:
            if file.lower().endswith('.json'):
                json_path = os.path.join(root, file)
                #print(f"Searching in {json_path}")
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        for obj in data:
                            if 'id' in obj and obj['id'] == image_name:
                                print(f"Answer: {obj.get('answer_key', 'No answer field')}")
                                return obj.get('answer_key', 'No answer field')
                except Exception as e:
                    print(f"Error reading {json_path}: {e}")

# Example usage
# image_name = '3c64b5ed-b124-4619-80a7-7d8652264024'  # Use the filename variable defined earlier
json_directory = 'D:\\pdf_processing_extraction\\dataset'  # TODO change json directory
# search_image_in_json(image_name, json_directory)

valid_converted_folder = os.path.join(directory_path, "valid_converted")

# Empty the folder with a warning
if os.path.exists(valid_converted_folder):
    print(f"Warning: The folder {valid_converted_folder} will be emptied.")
    for filename in os.listdir(valid_converted_folder):
        file_path = os.path.join(valid_converted_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")
os.makedirs(valid_converted_folder, exist_ok=True)

for root, _, files in os.walk(valid_folder):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            image_name = os.path.splitext(file)[0].split('_')[1]
            print("-----------------")
            print(image_name)
            answer = search_image_in_json(image_name, json_directory)
            print("-----------------")
            if answer:
                image = cv2.imread(os.path.join(root, file))
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                # Determine language from the parent folder TODO better mapping
                parent_folder = os.path.basename(os.path.dirname(root))
                if 'CRO' in parent_folder.lower():
                    lang = 'hrv'
                elif 'ITA' in parent_folder.lower():
                    lang = 'ita'
                elif 'SRP' in parent_folder.lower():
                    lang = 'srp'
                else:
                    lang = 'eng'
                
                text = pytesseract.image_to_string(gray, lang=lang)
                
                text_file_name = f"{image_name}_{answer}.txt"
                parent_dir_of_file = os.path.basename(root)
                destination_folder = os.path.join(valid_converted_folder, parent_dir_of_file)
                os.makedirs(destination_folder, exist_ok=True)
                
                text_file_path = os.path.join(destination_folder, text_file_name)
                with open(text_file_path, 'w', encoding='utf-8') as text_file:
                    text_file.write(text)
                print(f"Saved text to {text_file_path}")



In [ ]:
# Extracting only the answers from the converted files and saving them to a separate folder (valid_converted_only_answers)

valid_converted_only_answers_folder = os.path.join(directory_path, "valid_converted_only_answers")
if os.path.exists(valid_converted_only_answers_folder):
    print(f"Warning: The folder {valid_converted_only_answers_folder} will be emptied.")
    for filename in os.listdir(valid_converted_only_answers_folder):
        file_path = os.path.join(valid_converted_only_answers_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")
os.makedirs(valid_converted_only_answers_folder, exist_ok=True)

for root, _, files in os.walk(valid_converted_folder):
    for file in files:
        if file.lower().endswith('.txt'):

            file_id = file.split('_')[0]
            answer = file.split('_')[1].split('.')[0]
            

            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                content = f.read()
                _
                answer_pattern = f"{answer}."
                start_index = content.find(answer_pattern)
                if start_index != -1:
                    answer_text = content[start_index:].split('\n')[0]
                    print(f"ID: {file_id}, Answer: {answer_text}")

                    # Save everything before "A." pattern together with \n\n\n and the answer found
                    before_a_pattern = content.split("A.")[0] + "\n\n\n" + answer_text
                    
                    parent_folder_of_file = os.path.basename(root)
                    valid_converted_only_answers_folder = os.path.join(directory_path, "valid_converted_only_answers")
                    if not os.path.exists(valid_converted_only_answers_folder):
                        os.makedirs(valid_converted_only_answers_folder, exist_ok=True)
                        
                    output_file_path = os.path.join(valid_converted_only_answers_folder, parent_folder_of_file, f"{file_id}_answer.txt")
                    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
                    
                    with open(output_file_path, 'w', encoding='utf-8') as output_file:
                        output_file.write(before_a_pattern)
                    print(f"Saved answer to {output_file_path}")